# EDA: Анализ датасета повреждений автомобилей

Этот notebook содержит исследовательский анализ данных для системы обнаружения повреждений автомобилей.

## Содержание:
1. Загрузка и обзор данных
2. Анализ распределения классов
3. Анализ степеней повреждений
4. Проверка качества изображений
5. Детекция теней и бликов
6. Визуализация примеров
7. Анализ проблемных случаев

In [ ]:
import os
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from pathlib import Path
from collections import Counter, defaultdict
import warnings
warnings.filterwarnings('ignore')

# Добавляем путь к проекту
sys.path.append('..')

# Настройка отображения
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

## 1. Загрузка и обзор данных

In [ ]:
# Пути к данным
DATA_ROOT = "../data"
IMAGES_DIR = f"{DATA_ROOT}/curated"
ANNOTATIONS_FILE = f"{DATA_ROOT}/annotations/coco/instances.json"
SPLITS_DIR = f"{DATA_ROOT}/splits"

# Проверяем наличие файлов
print(f"Папка с изображениями: {os.path.exists(IMAGES_DIR)}")
print(f"Файл аннотаций: {os.path.exists(ANNOTATIONS_FILE)}")
print(f"Папка с разделениями: {os.path.exists(SPLITS_DIR)}")

In [ ]:
# Загружаем аннотации COCO
with open(ANNOTATIONS_FILE, 'r') as f:
    coco_data = json.load(f)

print("Структура данных COCO:")
for key in coco_data.keys():
    if isinstance(coco_data[key], list):
        print(f"  {key}: {len(coco_data[key])} элементов")
    else:
        print(f"  {key}: {type(coco_data[key])}")

In [ ]:
# Создаем DataFrame для удобного анализа
images_df = pd.DataFrame(coco_data['images'])
annotations_df = pd.DataFrame(coco_data['annotations'])
categories_df = pd.DataFrame(coco_data['categories'])

print("Основная статистика:")
print(f"Изображений: {len(images_df)}")
print(f"Аннотаций: {len(annotations_df)}")
print(f"Категорий: {len(categories_df)}")

# Показываем категории
print("\nКатегории повреждений:")
display(categories_df)

## 2. Анализ распределения классов

In [ ]:
# Подсчитываем количество аннотаций по категориям
category_counts = annotations_df['category_id'].value_counts().sort_index()

# Добавляем названия категорий
category_names = {cat['id']: cat['name'] for cat in coco_data['categories']}
category_counts_named = category_counts.copy()
category_counts_named.index = [category_names[idx] for idx in category_counts.index]

# Визуализация распределения классов
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Столбчатая диаграмма
category_counts_named.plot(kind='bar', ax=ax1, color='skyblue')
ax1.set_title('Распределение категорий повреждений')
ax1.set_xlabel('Категория')
ax1.set_ylabel('Количество аннотаций')
ax1.tick_params(axis='x', rotation=45)

# Круговая диаграмма
category_counts_named.plot(kind='pie', ax=ax2, autopct='%1.1f%%')
ax2.set_title('Процентное распределение категорий')
ax2.set_ylabel('')

plt.tight_layout()
plt.show()

print("Статистика по категориям:")
for cat_id, count in category_counts.items():
    cat_name = category_names[cat_id]
    percentage = count / len(annotations_df) * 100
    print(f"{cat_name}: {count} ({percentage:.1f}%)")

## 3. Анализ степеней повреждений

In [ ]:
# Создаем маппинг степеней повреждений
damage_levels = {cat['id']: cat.get('damage_level', 0) for cat in coco_data['categories']}
damage_level_names = {
    0: "Нет повреждений",
    1: "Легкие повреждения", 
    2: "Умеренные повреждения",
    3: "Серьезные повреждения"
}

# Добавляем уровни повреждений к аннотациям
annotations_df['damage_level'] = annotations_df['category_id'].map(damage_levels)

# Анализируем распределение по уровням
damage_distribution = annotations_df['damage_level'].value_counts().sort_index()
damage_distribution_named = damage_distribution.copy()
damage_distribution_named.index = [damage_level_names[idx] for idx in damage_distribution.index]

# Визуализация
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
colors = ['green', 'yellow', 'orange', 'red']
damage_distribution_named.plot(kind='bar', ax=ax, color=colors[:len(damage_distribution)])
ax.set_title('Распределение по степеням повреждений')
ax.set_xlabel('Степень повреждения')
ax.set_ylabel('Количество аннотаций')
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("Распределение по критичности:")
for level, count in damage_distribution.items():
    level_name = damage_level_names[level]
    percentage = count / len(annotations_df) * 100
    print(f"{level_name}: {count} ({percentage:.1f}%)")

## 4. Анализ изображений

In [ ]:
# Анализ размеров изображений
image_sizes = []
aspect_ratios = []
corrupted_images = []

for _, img_info in images_df.iterrows():
    img_path = os.path.join(IMAGES_DIR, img_info['file_name'])
    
    if os.path.exists(img_path):
        try:
            width, height = img_info['width'], img_info['height']
            image_sizes.append((width, height))
            aspect_ratios.append(width / height)
        except Exception as e:
            corrupted_images.append(img_info['file_name'])
    else:
        print(f"Файл не найден: {img_info['file_name']}")

# Статистика размеров
widths, heights = zip(*image_sizes) if image_sizes else ([], [])

print(f"Анализировано изображений: {len(image_sizes)}")
print(f"Поврежденных файлов: {len(corrupted_images)}")

if image_sizes:
    print(f"\nРазмеры изображений:")
    print(f"  Ширина: {min(widths)} - {max(widths)} (среднее: {np.mean(widths):.0f})")
    print(f"  Высота: {min(heights)} - {max(heights)} (среднее: {np.mean(heights):.0f})")
    print(f"  Соотношение сторон: {min(aspect_ratios):.2f} - {max(aspect_ratios):.2f}")

In [ ]:
# Визуализация распределения размеров
if image_sizes:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Ширина
    axes[0,0].hist(widths, bins=30, alpha=0.7, color='blue')
    axes[0,0].set_title('Распределение ширины изображений')
    axes[0,0].set_xlabel('Ширина (пиксели)')
    axes[0,0].set_ylabel('Количество')
    
    # Высота
    axes[0,1].hist(heights, bins=30, alpha=0.7, color='green')
    axes[0,1].set_title('Распределение высоты изображений')
    axes[0,1].set_xlabel('Высота (пиксели)')
    axes[0,1].set_ylabel('Количество')
    
    # Соотношение сторон
    axes[1,0].hist(aspect_ratios, bins=30, alpha=0.7, color='orange')
    axes[1,0].set_title('Распределение соотношения сторон')
    axes[1,0].set_xlabel('Ширина / Высота')
    axes[1,0].set_ylabel('Количество')
    
    # Scatter plot размеров
    axes[1,1].scatter(widths, heights, alpha=0.6)
    axes[1,1].set_title('Соотношение ширины и высоты')
    axes[1,1].set_xlabel('Ширина')
    axes[1,1].set_ylabel('Высота')
    
    plt.tight_layout()
    plt.show()

## 5. Детекция теней и бликов

In [ ]:
def analyze_lighting_conditions(image_path):
    """Анализирует условия освещения на изображении"""
    try:
        image = cv2.imread(image_path)
        if image is None:
            return None
        
        # Конвертируем в HSV
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        
        # Анализ яркости (V канал)
        brightness = hsv[:,:,2]
        mean_brightness = np.mean(brightness)
        std_brightness = np.std(brightness)
        
        # Детекция теней (низкая яркость)
        shadow_mask = brightness < (mean_brightness * 0.5)
        shadow_percentage = np.sum(shadow_mask) / shadow_mask.size * 100
        
        # Детекция бликов (высокая яркость)
        glare_mask = brightness > 240
        glare_percentage = np.sum(glare_mask) / glare_mask.size * 100
        
        return {
            'mean_brightness': mean_brightness,
            'brightness_std': std_brightness,
            'shadow_percentage': shadow_percentage,
            'glare_percentage': glare_percentage,
            'has_shadows': shadow_percentage > 15,  # более 15% теней
            'has_glare': glare_percentage > 5       # более 5% бликов
        }
    except Exception as e:
        return None

# Анализируем первые 20 изображений
lighting_analysis = []
sample_size = min(20, len(images_df))

for _, img_info in images_df.head(sample_size).iterrows():
    img_path = os.path.join(IMAGES_DIR, img_info['file_name'])
    if os.path.exists(img_path):
        analysis = analyze_lighting_conditions(img_path)
        if analysis:
            analysis['filename'] = img_info['file_name']
            lighting_analysis.append(analysis)

if lighting_analysis:
    lighting_df = pd.DataFrame(lighting_analysis)
    
    print(f"Анализ освещения ({len(lighting_df)} изображений):")
    print(f"Средняя яркость: {lighting_df['mean_brightness'].mean():.1f} ± {lighting_df['mean_brightness'].std():.1f}")
    print(f"Изображений с тенями: {lighting_df['has_shadows'].sum()} ({lighting_df['has_shadows'].mean()*100:.1f}%)")
    print(f"Изображений с бликами: {lighting_df['has_glare'].sum()} ({lighting_df['has_glare'].mean()*100:.1f}%)")
    
    # Визуализация
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Распределение яркости
    axes[0].hist(lighting_df['mean_brightness'], bins=15, alpha=0.7, color='yellow')
    axes[0].set_title('Распределение средней яркости')
    axes[0].set_xlabel('Средняя яркость')
    axes[0].set_ylabel('Количество изображений')
    
    # Процент теней
    axes[1].hist(lighting_df['shadow_percentage'], bins=15, alpha=0.7, color='darkblue')
    axes[1].set_title('Процент теней на изображениях')
    axes[1].set_xlabel('Процент теней')
    axes[1].set_ylabel('Количество изображений')
    
    # Процент бликов
    axes[2].hist(lighting_df['glare_percentage'], bins=15, alpha=0.7, color='white', edgecolor='black')
    axes[2].set_title('Процент бликов на изображениях')
    axes[2].set_xlabel('Процент бликов')
    axes[2].set_ylabel('Количество изображений')
    
    plt.tight_layout()
    plt.show()

## 6. Визуализация примеров данных

In [ ]:
def draw_bbox_on_image(image, bbox, label, color=(255, 0, 0)):
    """Рисует bounding box на изображении"""
    x, y, w, h = bbox
    cv2.rectangle(image, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
    cv2.putText(image, label, (int(x), int(y - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return image

def visualize_sample_images(num_samples=6):
    """Визуализирует примеры изображений с аннотациями"""
    
    # Выбираем изображения с аннотациями
    images_with_annotations = annotations_df['image_id'].unique()
    selected_images = np.random.choice(images_with_annotations, 
                                     min(num_samples, len(images_with_annotations)), 
                                     replace=False)
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255)]
    
    for i, img_id in enumerate(selected_images):
        if i >= len(axes):
            break
            
        # Получаем информацию об изображении
        img_info = images_df[images_df['id'] == img_id].iloc[0]
        img_path = os.path.join(IMAGES_DIR, img_info['file_name'])
        
        if not os.path.exists(img_path):
            continue
            
        # Загружаем изображение
        image = cv2.imread(img_path)
        if image is None:
            continue
            
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Получаем аннотации для этого изображения
        img_annotations = annotations_df[annotations_df['image_id'] == img_id]
        
        # Рисуем bounding boxes
        for _, ann in img_annotations.iterrows():
            bbox = ann['bbox']
            cat_id = ann['category_id']
            cat_name = category_names[cat_id]
            damage_level = damage_levels[cat_id]
            
            label = f"{cat_name} (L{damage_level})"
            color = colors[cat_id % len(colors)]
            
            image = draw_bbox_on_image(image, bbox, label, color)
        
        # Отображаем
        axes[i].imshow(image)
        axes[i].set_title(f"{img_info['file_name']}\n({len(img_annotations)} аннотаций)")
        axes[i].axis('off')
    
    # Скрываем пустые субплоты
    for j in range(len(selected_images), len(axes)):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()

# Визуализируем примеры
visualize_sample_images(6)

## 7. Анализ размеров аннотаций

In [ ]:
# Анализируем размеры bounding boxes
bbox_areas = []
bbox_aspect_ratios = []
bbox_sizes_by_category = defaultdict(list)

for _, ann in annotations_df.iterrows():
    x, y, w, h = ann['bbox']
    area = w * h
    aspect_ratio = w / h if h > 0 else 0
    
    bbox_areas.append(area)
    bbox_aspect_ratios.append(aspect_ratio)
    
    cat_name = category_names[ann['category_id']]
    bbox_sizes_by_category[cat_name].append(area)

# Статистика
print("Статистика размеров аннотаций:")
print(f"Средняя площадь: {np.mean(bbox_areas):.0f} пикселей²")
print(f"Медианная площадь: {np.median(bbox_areas):.0f} пикселей²")
print(f"Мин/макс площадь: {min(bbox_areas):.0f} - {max(bbox_areas):.0f}")
print(f"Среднее соотношение сторон: {np.mean(bbox_aspect_ratios):.2f}")

# Визуализация
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Распределение площадей
axes[0,0].hist(bbox_areas, bins=30, alpha=0.7, color='purple')
axes[0,0].set_title('Распределение площадей аннотаций')
axes[0,0].set_xlabel('Площадь (пиксели²)')
axes[0,0].set_ylabel('Количество')
axes[0,0].set_yscale('log')

# Соотношение сторон
axes[0,1].hist(bbox_aspect_ratios, bins=30, alpha=0.7, color='brown')
axes[0,1].set_title('Соотношение сторон аннотаций')
axes[0,1].set_xlabel('Ширина / Высота')
axes[0,1].set_ylabel('Количество')

# Boxplot размеров по категориям
category_names_list = list(bbox_sizes_by_category.keys())
category_areas_list = [bbox_sizes_by_category[cat] for cat in category_names_list]

axes[1,0].boxplot(category_areas_list, labels=category_names_list)
axes[1,0].set_title('Размеры аннотаций по категориям')
axes[1,0].set_ylabel('Площадь (пиксели²)')
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].set_yscale('log')

# Среднее количество аннотаций на изображение
annotations_per_image = annotations_df.groupby('image_id').size()
axes[1,1].hist(annotations_per_image, bins=range(1, max(annotations_per_image)+2), 
               alpha=0.7, color='teal')
axes[1,1].set_title('Количество аннотаций на изображение')
axes[1,1].set_xlabel('Количество аннотаций')
axes[1,1].set_ylabel('Количество изображений')

plt.tight_layout()
plt.show()

print(f"\nСреднее количество аннотаций на изображение: {annotations_per_image.mean():.2f}")

## 8. Анализ проблемных случаев

In [ ]:
# Находим потенциально проблемные случаи
print("Потенциальные проблемы в данных:")

# 1. Очень маленькие аннотации (возможно, ошибки разметки)
small_annotations = annotations_df[annotations_df.apply(
    lambda x: x['bbox'][2] * x['bbox'][3] < 100, axis=1
)]
print(f"1. Очень маленькие аннотации (< 100 пикс²): {len(small_annotations)}")

# 2. Очень большие аннотации (возможно, неточная разметка)
large_annotations = annotations_df[annotations_df.apply(
    lambda x: x['bbox'][2] * x['bbox'][3] > np.percentile(bbox_areas, 95), axis=1
)]
print(f"2. Очень большие аннотации (> 95 перцентиль): {len(large_annotations)}")

# 3. Изображения без аннотаций
images_with_annotations = set(annotations_df['image_id'].unique())
all_images = set(images_df['id'].unique())
images_without_annotations = all_images - images_with_annotations
print(f"3. Изображения без аннотаций: {len(images_without_annotations)}")

# 4. Изображения с большим количеством аннотаций
images_many_annotations = annotations_per_image[annotations_per_image > np.percentile(annotations_per_image, 95)]
print(f"4. Изображения с очень большим количеством аннотаций: {len(images_many_annotations)}")

# 5. Дисбаланс классов
min_class_count = category_counts.min()
max_class_count = category_counts.max()
imbalance_ratio = max_class_count / min_class_count
print(f"5. Соотношение дисбаланса классов: {imbalance_ratio:.1f}:1")

# Рекомендации
print("\nРекомендации для улучшения датасета:")
if imbalance_ratio > 5:
    print("- Сильный дисбаланс классов. Рекомендуется аугментация редких классов или взвешенный loss")
if len(small_annotations) > len(annotations_df) * 0.05:
    print("- Много мелких аннотаций. Проверьте качество разметки")
if len(images_without_annotations) > len(all_images) * 0.1:
    print("- Много изображений без аннотаций. Убедитесь, что это негативные примеры")
if lighting_df['has_shadows'].mean() > 0.3:
    print("- Много изображений с тенями. Используйте специальную предобработку")
if lighting_df['has_glare'].mean() > 0.2:
    print("- Много изображений с бликами. Рекомендуется фильтрация бликов")

## 9. Выводы и рекомендации

In [ ]:
# Итоговая сводка
print("=" * 60)
print("ИТОГОВЫЙ ОТЧЕТ ПО EDA")
print("=" * 60)

print(f"\n📊 ОСНОВНАЯ СТАТИСТИКА:")
print(f"   • Всего изображений: {len(images_df)}")
print(f"   • Всего аннотаций: {len(annotations_df)}")
print(f"   • Категорий повреждений: {len(categories_df)}")
print(f"   • Среднее аннотаций на изображение: {annotations_per_image.mean():.2f}")

print(f"\n🎯 РАСПРЕДЕЛЕНИЕ КЛАССОВ:")
for cat_id, count in category_counts.items():
    cat_name = category_names[cat_id]
    percentage = count / len(annotations_df) * 100
    print(f"   • {cat_name}: {count} ({percentage:.1f}%)")

print(f"\n⚠️  КРИТИЧНОСТЬ ПОВРЕЖДЕНИЙ:")
for level, count in damage_distribution.items():
    level_name = damage_level_names[level]
    percentage = count / len(annotations_df) * 100
    print(f"   • {level_name}: {count} ({percentage:.1f}%)")

if lighting_analysis:
    print(f"\n💡 УСЛОВИЯ ОСВЕЩЕНИЯ:")
    print(f"   • Средняя яркость: {lighting_df['mean_brightness'].mean():.1f}")
    print(f"   • Изображений с тенями: {lighting_df['has_shadows'].sum()}/{len(lighting_df)} ({lighting_df['has_shadows'].mean()*100:.1f}%)")
    print(f"   • Изображений с бликами: {lighting_df['has_glare'].sum()}/{len(lighting_df)} ({lighting_df['has_glare'].mean()*100:.1f}%)")

print(f"\n🔧 РЕКОМЕНДАЦИИ ДЛЯ ОБУЧЕНИЯ:")
print(f"   1. Использовать взвешенный loss для балансировки классов")
print(f"   2. Применить аугментации для редких классов")
print(f"   3. Реализовать предобработку теней и бликов")
print(f"   4. Использовать 'кнут и пряник' для критичных классов (уровень 3)")
print(f"   5. Добавить hard negative mining для сложных примеров")

print(f"\n📝 СЛЕДУЮЩИЕ ШАГИ:")
print(f"   • Проверить и исправить мелкие/большие аннотации")
print(f"   • Добавить больше данных для редких классов")
print(f"   • Валидировать негативные примеры (без повреждений)")
print(f"   • Протестировать алгоритмы предобработки теней")
print("=" * 60)

## 10. Сохранение результатов анализа

In [ ]:
# Сохраняем результаты анализа
analysis_results = {
    'dataset_stats': {
        'total_images': len(images_df),
        'total_annotations': len(annotations_df),
        'categories_count': len(categories_df),
        'avg_annotations_per_image': float(annotations_per_image.mean())
    },
    'class_distribution': {category_names[k]: int(v) for k, v in category_counts.items()},
    'damage_level_distribution': {damage_level_names[k]: int(v) for k, v in damage_distribution.items()},
    'bbox_stats': {
        'mean_area': float(np.mean(bbox_areas)),
        'median_area': float(np.median(bbox_areas)),
        'min_area': float(min(bbox_areas)),
        'max_area': float(max(bbox_areas)),
        'mean_aspect_ratio': float(np.mean(bbox_aspect_ratios))
    },
    'quality_issues': {
        'small_annotations': len(small_annotations),
        'large_annotations': len(large_annotations),
        'images_without_annotations': len(images_without_annotations),
        'class_imbalance_ratio': float(imbalance_ratio)
    }
}

if lighting_analysis:
    analysis_results['lighting_conditions'] = {
        'mean_brightness': float(lighting_df['mean_brightness'].mean()),
        'images_with_shadows': int(lighting_df['has_shadows'].sum()),
        'images_with_glare': int(lighting_df['has_glare'].sum()),
        'shadow_percentage': float(lighting_df['has_shadows'].mean() * 100),
        'glare_percentage': float(lighting_df['has_glare'].mean() * 100)
    }

# Сохраняем в JSON
output_dir = Path("../experiments/eda_results")
output_dir.mkdir(parents=True, exist_ok=True)

with open(output_dir / "eda_analysis.json", 'w') as f:
    json.dump(analysis_results, f, indent=2, ensure_ascii=False)

print(f"Результаты анализа сохранены в {output_dir / 'eda_analysis.json'}")

# Сохраняем DataFrame для дальнейшего анализа
annotations_df.to_csv(output_dir / "annotations_enhanced.csv", index=False)
if lighting_analysis:
    lighting_df.to_csv(output_dir / "lighting_analysis.csv", index=False)

print("EDA завершен! 🎉")